In [1]:
import os,time,sys
import ghtest.param_suggestor as ps
from ghtest import make_test, scan, suggest, write_module
from ghtest.tests_writer import TestArtifact
#ps.VB=1

In [2]:
# set print behavior
from IPython.display import display, Javascript
display(Javascript("""
document.querySelectorAll('.jp-OutputArea-child').forEach(e => e.style.maxHeight = 'none');
"""))

<IPython.core.display.Javascript object>

In [71]:
# input
cassette_dir='tests/cassettes'
test_dir='testdata_tests/'
#src_folder='../temp_folder/src/apigh/'
src_folder='testdata'
#test_folder='../temp_folder/tests/'

In [72]:
# scan 
scs=scan(src_folder)
#sct=scan(test_folder)len(scs)#, len(sct)

In [73]:
# filter non-api and test methods
funcs=[f for f in scs if not (f.qualname.startswith('_') or f.qualname.startswith('test_'))]
#funcs=scs
for func in funcs:
    print(func.qualname )

get_commits
list_repos
create_repo
delete_repo
get_repo_data


In [74]:
# check functions
for func in funcs:
    print(func.qualname)
    [p for p in func.parameters if p.name=='user']
    {k:v for k,v in func.module_globals.items() if 'user' in k}
    [c for c in func.sample_calls if any('user' in k for k in c)]
    [c for c in func.module_call_values if any('user' in k for k in c)]
    {k:v for k,v in func.module_param_values.items() if 'user' in k}
    
    #func.crud_role
    #func.crud_resource
    #[[(p.name, p.default) for p in f.parameters if 'user' in p.name] for f in func.module_functions]
    #[f for f in [[(p.name, p.default) for p in f.parameters if p.default and 'k4144' in p.default] for f in func.module_functions] if f]
    vars(func).keys()
    keys=['module', 'qualname', 'filepath', 'lineno', 'parameters', 'returns', 'docstring', 'module_globals', 'sample_calls', 'module_call_values', 'module_param_values', 'crud_role', 'crud_resource', 'module_functions']
    for k in keys:
        s=eval(f'func.{k}')
        if 'k4144' in str(s).lower():
            print(k)
    if 'k4144' in str(func).lower():
        print('k4144')

get_commits
list_repos
create_repo
delete_repo
get_repo_data


In [75]:
sps=[]
for func in funcs:
    sp=suggest(func,test_functions=None, literal_only=True)
    print(sp.module, sp.qualname)
    for ps in sp.param_sets:
        print(ps)
    sps.append(sp)

gh_api get_commits
{'name': 'coron', 'vb': 1, 'dry_run': False}
{'name': 'coron6', 'vb': 1, 'dry_run': False}
{'name': 'coron234', 'vb': 1, 'dry_run': False}
{'name': 'example', 'max_commits': 50, 'return_all': True, 'vb': 1}
{'name': 'coron', 'max_commits': 'max', 'return_all': True, 'vb': 3}
{'name': 'coron'}
{'name': 'coron', 'user': False}
{'name': 'coron', 'user': True}
{'name': 'coron', 'max_commits': 50}
{'name': 'coron', 'dry_run': False}
{'name': 'coron', 'dry_run': True}
{'name': 'coron', 'return_all': False}
{'name': 'coron', 'return_all': True}
{'name': 'coron', 'vb': 0}
{'name': 'coron', 'vb': 1}
{'name': 'coron', 'vb': 2}
gh_api list_repos
{'max_repos': 30, 'vb': 1}
{}
{'user': False}
{'user': True}
{'vb': 0}
{'vb': 1}
{'max_repos': 30}
gh_api create_repo
{'name': 'coron', 'vb': 1, 'dry_run': False}
{'name': 'coron6', 'vb': 2, 'dry_run': False}
{'name': 'coron'}
{'name': 'coron', 'user': 'gh4144'}
{'name': 'coron', 'description': 'simple repo'}
{'name': 'coron', 'private'

In [76]:
# change param
# for security, we change 'coron' to coron7
param_update={'name':'coron7'}
for sp in sps:
    updated_param_set=[]
    for op in sp.param_sets:
        for k,v in param_update.items():
            if k in op:
                op=op|param_update
        updated_param_set.append(op)
    sp.param_sets=updated_param_set
    print(sp.qualname)
    print(sp.param_sets)
    print()

get_commits
[{'name': 'coron7', 'vb': 1, 'dry_run': False}, {'name': 'coron7', 'vb': 1, 'dry_run': False}, {'name': 'coron7', 'vb': 1, 'dry_run': False}, {'name': 'coron7', 'max_commits': 50, 'return_all': True, 'vb': 1}, {'name': 'coron7', 'max_commits': 'max', 'return_all': True, 'vb': 3}, {'name': 'coron7'}, {'name': 'coron7', 'user': False}, {'name': 'coron7', 'user': True}, {'name': 'coron7', 'max_commits': 50}, {'name': 'coron7', 'dry_run': False}, {'name': 'coron7', 'dry_run': True}, {'name': 'coron7', 'return_all': False}, {'name': 'coron7', 'return_all': True}, {'name': 'coron7', 'vb': 0}, {'name': 'coron7', 'vb': 1}, {'name': 'coron7', 'vb': 2}]

list_repos
[{'max_repos': 30, 'vb': 1}, {}, {'user': False}, {'user': True}, {'vb': 0}, {'vb': 1}, {'max_repos': 30}]

create_repo
[{'name': 'coron7', 'vb': 1, 'dry_run': False}, {'name': 'coron7', 'vb': 2, 'dry_run': False}, {'name': 'coron7'}, {'name': 'coron7', 'user': 'gh4144'}, {'name': 'coron7', 'description': 'simple repo'}, {

In [80]:
# make tests
remove_cassette=True

gts=[]
for sp in sps:
    gt=make_test(suggestion=sp, cassette_dir=cassette_dir)
    gts.append(gt)
    cp=gt.cassette_path
    if remove_cassette and os.path.isfile(cp):
        os.remove(cp)

In [81]:
# remove all previous tests 
remove_tests=True
import shutil
for file in os.listdir(test_dir):
    path=os.path.join(test_dir, file)
    if os.path.isfile(path):
        os.remove(path)
    elif os.path.isdir(path):
        shutil.rmtree(path)

In [82]:
trs=[]
for gt in gts:
    try:
        tr=gt.test_callable()
        trs.append(tr)
    except Exception as e:
        print(str(e))

Planned CRUD scenario for resource 'repo' as 'repl1yky7':
  - get_repo_data(name='repl1yky7', dry_run=False)
  - create_repo(name='repl1yky7', dry_run=False)
  - get_repo_data(name='repl1yky7', dry_run=False)
  - delete_repo(name='repl1yky7', dry_run=False, force=True)
  - get_repo_data(name='repl1yky7', dry_run=False)
Note: Set GHTEST_ASSUME_SAFE=1 to skip confirmations before running this scenario.
Proceed with the full sequence? [y/N]:  y
Function delete_repo in testdata/gh_api.py may perform destructive actions.
Proceed with executing auto-generated tests? [y/N]:  y


In [84]:
artifacts=[]
rrs=[]
for sp,tr in zip(sps, trs):
    artifacts.append(TestArtifact(suggestion=sp, run=tr))
r=write_module(artifacts=artifacts, output_dir=test_dir)
r

TestWriterResult(test_modules=[PosixPath('testdata_tests/test_generated_0.py'), PosixPath('testdata_tests/test_generated_1.py'), PosixPath('testdata_tests/test_generated_2.py'), PosixPath('testdata_tests/test_generated_3.py'), PosixPath('testdata_tests/test_generated_4.py'), PosixPath('testdata_tests/test_generated_5.py')], scenario_modules=[PosixPath('testdata_tests/scenarios/scenario_repo_0.py')])

In [12]:
for case in tr.cases:
    print('#'*30)
    d=vars(case)
    print(case.target)
    for var in ('params', 'return_value', 'printed', 'exception'):
        r=d[var]
        if isinstance(r,list):
            print(var, len(r))
        else:
            print(var, r)

##############################
create_repo
params {'name': 'coron', 'vb': 1, 'dry_run': False}
return_value False
printed error: name already exists on this account

exception None
##############################
create_repo
params {'name': 'coron6', 'vb': 2, 'dry_run': False}
return_value False
printed {'Accept': 'application/vnd.github+json', 'X-GitHub-Api-Version': '2022-11-28', 'Content-Type': 'application/x-www-form-urlencoded', 'authorization': 'token github_pa..mU 99'}
{"name": "coron6", "description": "simple repo allow_del_by_api=true ", "homepage": "https://github.com/gh4144/coron6", "private": true, "is_template": false}
preparing request
status 422
error: name already exists on this account

exception None
##############################
create_repo
params {'name': 'coron'}
return_value None
printed 
exception None
##############################
create_repo
params {'name': 'coron', 'user': 'gh4144'}
return_value None
printed 
exception None
##############################
crea

In [12]:
vars(case).keys()

dict_keys(['target', 'params', 'return_value', 'printed', 'exception'])

In [13]:
tr1=gt.test_callable()

Planned CRUD scenario for resource 'repo' as 'repi0iqqo':
  - get_repo_data(name='repi0iqqo', dry_run=False)
  - create_repo(name='repi0iqqo', dry_run=False)
  - get_repo_data(name='repi0iqqo', dry_run=False)
  - delete_repo(name='repi0iqqo', dry_run=False, force=True)
  - get_repo_data(name='repi0iqqo', dry_run=False)
Note: Set GHTEST_ASSUME_SAFE=1 to skip confirmations before running this scenario.
Proceed with the full sequence? [y/N]:  y


In [29]:
len(tr1.cases)

17

In [14]:
def check(tr1, tr2, vb=0):
    assert tr1.cassette_path==tr2.cassette_path
    cs1,cs2=tr1.cases,tr2.cases
    assert len(cs1)==len(cs2)
    for c1,c2 in zip(cs1,cs2):
        d1, d2=vars(c1), vars(c2)
        for var in ('params', 'return_value', 'printed', 'exception'):
            r1,r2=d1[var], d2[var]
            if isinstance(r1,list):
                if vb:print(var, len(r1))
            else:
                if vb:print(var, r1)
            if isinstance(r2,list):
                if vb:print(var, len(r2))
            else:
                if vb:print(var, r2)
            if isinstance(r1,list|dict|tuple) and isinstance(r2,list|dict|tuple):
                if vb:print('assert len')
                assert len(r1)==len(r2)
                if vb:print('assert identity')
                assert r1==r2

In [15]:
check(tr,tr1)

In [16]:
def print_value(v):
    if v is None:print(v)
    if isinstance(v,dict|list|tuple):
        print(f'len: {len(v)}')
    if isinstance(v,dict):
        print(f'keys: {v.keys()}')

In [17]:
for case in tr.cases:
    #print('#'*30)
    #print(case.target)
    #print(case.params)
    #print(case.printed)
    #print(case.exception)
    #print_value(case.return_value)
    
    #print()
    pass

In [19]:
vars(tr).keys()

dict_keys(['cassette_path', 'cases'])

In [20]:
print(gt.source)

import importlib
import io
import os
from contextlib import redirect_stdout
from dataclasses import dataclass
from typing import Any, Dict, List, Optional

import vcr


@dataclass
class TestCaseResult:
    target: str
    params: Dict[str, Any]
    return_value: Any
    printed: str
    exception: Optional[BaseException]


@dataclass
class TestRunWithCassette:
    cassette_path: str
    cases: List[TestCaseResult]


def _import_module_from_path(path: str, module_name: str = None) -> 'ModuleType':
    """
    Import a Python module directly from its file path.
    Returns the loaded module object.
    """
    if module_name is None:
        # Generate a unique name from the filename
        import os, uuid
        base = os.path.splitext(os.path.basename(path))[0]
        module_name = f"{base}_{uuid.uuid4().hex}"

    spec = importlib.util.spec_from_file_location(module_name, path)
    if spec is None or spec.loader is None:
        raise ImportError(f"cannot import module from {path}"

In [32]:
param_sets = [{'name': 'coron', 'vb': 1, 'dry_run': False}, {'name': 'coron6', 'vb': 2, 'dry_run': False}, {'name': 'coron'}, {'name': 'coron', 'user': 'gh4144'}, {'name': 'coron', 'description': 'simple repo'}, {'name': 'coron', 'private': True}, {'name': 'coron', 'is_template': False}, {'name': 'coron', 'dry_run': False}, {'name': 'coron', 'dry_run': True}, {'name': 'coron', 'vb': 0}, {'name': 'coron', 'vb': 1}, {'name': 'coron', 'vb': 2}]
param_sets

[{'name': 'coron', 'vb': 1, 'dry_run': False},
 {'name': 'coron6', 'vb': 2, 'dry_run': False},
 {'name': 'coron'},
 {'name': 'coron', 'user': 'gh4144'},
 {'name': 'coron', 'description': 'simple repo'},
 {'name': 'coron', 'private': True},
 {'name': 'coron', 'is_template': False},
 {'name': 'coron', 'dry_run': False},
 {'name': 'coron', 'dry_run': True},
 {'name': 'coron', 'vb': 0},
 {'name': 'coron', 'vb': 1},
 {'name': 'coron', 'vb': 2}]

In [37]:
for c in tr.cases:
    print(c.target, c.params)

create_repo {'name': 'coron', 'vb': 1, 'dry_run': False}
create_repo {'name': 'coron6', 'vb': 2, 'dry_run': False}
create_repo {'name': 'coron'}
create_repo {'name': 'coron', 'user': 'gh4144'}
create_repo {'name': 'coron', 'description': 'simple repo'}
create_repo {'name': 'coron', 'private': True}
create_repo {'name': 'coron', 'is_template': False}
create_repo {'name': 'coron', 'dry_run': False}
create_repo {'name': 'coron', 'dry_run': True}
create_repo {'name': 'coron', 'vb': 0}
create_repo {'name': 'coron', 'vb': 1}
create_repo {'name': 'coron', 'vb': 2}
get_repo_data {'name': 'repi0iqqo', 'dry_run': False}
create_repo {'name': 'repi0iqqo', 'dry_run': False}
get_repo_data {'name': 'repi0iqqo', 'dry_run': False}
delete_repo {'name': 'repi0iqqo', 'dry_run': False, 'force': True}
get_repo_data {'name': 'repi0iqqo', 'dry_run': False}
